In [24]:
import os
import openai
from tqdm.auto import tqdm

In [3]:
from openai import OpenAI

In [4]:
client = OpenAI()

In [5]:
response = client.chat.completions.create(
model='gpt-4o-mini',
messages=[{"role":"user", "content":"is too late to joing the courcse?"}]
)

In [6]:
response.choices[0].message.content

"Whether it is too late to join a course depends on several factors, such as the specific course, its start date, and the institution or platform offering it. If the course has already started, some programs may allow late enrollment, while others may not. I recommend checking the course's website or contacting the instructor or administration directly for the most accurate information regarding enrollment deadlines. If you provide more details about the course, I might be able to help you further!"

# Homework Secction

## Q1. Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

### What's the version.build_hash value?

In [ ]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('htt://localhost:9200')


In [16]:
es_client.info()['version']['build_hash']

'42f05b9372a9a4a470db3b52817899b99a76ee73'

### Getting the data

In [17]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [19]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Q2. Indexing the data

## Q2. Indexing the data

### Which function do you use for adding your data to elastic? 
The answer is  `index`

In [25]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [26]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

# Q3. Searching

## query "How do execute a command on a Kubernetes pod?".

### What's the score for the top ranking result?

* The answer is  `'_score': 44.50556,`



In [48]:
query = 'How do execute a command on a Kubernetes pod?'

In [50]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }


In [51]:
response = es_client.search(index=index_name, body=search_query)


In [52]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'oiqTcJcBrJM5Tilwrsaf',
  '_score': 44.50556,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'MSqTcJcBrJM5Tilws8cn',
  '_score': 35.433445,
  '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
   'section': '10. Kubernetes and TensorFlow Serving',
   'question': 'Kubernetes-dashboard',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'wiqTcJcBrJM5Tilwr8aJ',
  '_score': 33.70974,
  '_source': {'text': 'You 

# Q4. Filtering

## Now ask a different question: "How do copy a file to a Docker container?".

### Return 3 results. What's the 3rd question returned by the search engine?

* The answer is  `How do I copy files from a different folder into docker container’s working directory?`

In [71]:
query2 = 'How do copy a file to a Docker container?'

In [72]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query2,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

In [73]:
response = es_client.search(index=index_name, body=search_query)

In [74]:
response['hits']['hits'][:3]

[{'_index': 'course-questions',
  '_id': 'oiqTcJcBrJM5Tilwrsaf',
  '_score': 55.040073,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'wSqTcJcBrJM5Tilwr8aC',
  '_score': 50.016533,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file

# Q5. Building a prompt

## What's the length of the resulting prompt? (use the len function)

* The answer is  `?`

In [75]:
query2

'How do copy a file to a Docker container?'

In [ ]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [85]:
result_docs = []
    
for hit in response['hits']['hits']:
   result_docs.append(hit['_source'])


In [79]:
len(result_docs)

5

In [99]:
context = ""

for doc in result_docs:
    context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"

In [96]:
prompt = prompt_template.format(question=query2, context=context).strip()


In [97]:
len(prompt)

2154

In [98]:
print(prompt)

You're a course teaching assitstant. Anwser the QUESTION base on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION

QUESTION: How do copy a file to a Docker container?

CONTEXT: 
Q: How do I debug a docker container?A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container